In [90]:
#Import relevant packages and functions

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  precision_score, recall_score,f1_score, brier_score_loss,roc_curve, auc, accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import csv
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pylab as pl

## Data Loading

The unit of analysis are the 3000 ft. x 3000 ft. squares partitioned in ArcGIS. 



In [129]:
#Set training equal to filename of dataset

training = 'Illegal_Construction_Dataset.csv'

In [149]:
#Read the dataset and split into X-vars and Y-var

model_train_data = pd.read_csv(training)
model_train_X = model_train_data[['X1', 'X2', 'X7', 'X21', 'X25']]
model_train_Y = model_train_data['Y_dummy']

The dataset is made up of 932 partitions, and 458 of the partitions contained instances of ECB summonses where work without a permit occurred and the case was not dismissed. Those cases that were not dismissed are identified to be the confirmed illegal construction cases in our dataset.

In [161]:
model_train_data['Y_dummy'].describe()

count    932.000000
mean       0.491416
std        0.500195
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Y_dummy, dtype: float64

In [160]:
model_train_data['Y_dummy'].sum()

458.0

# Fitting the Model

In [163]:
#Logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(model_train_X, model_train_Y)

# check the accuracy on the training set
model.score(model_train_X, model_train_Y)

0.55364806866952787

In [164]:
model_train_Y.mean()

0.49141630901287553

The model performs just slightly better than 50%, and this could be due to a downward bias in the data since illegal construction is not always reported, especially in low-income communities because the illegal units provide housing at below-market rents.

In [166]:
# examine the coefficients
pd.DataFrame(zip(model_train_X.columns, np.transpose(model.coef_)))

,0,1
0,X1,[0.000626932380797]
1,X2,[0.00589029720553]
2,X7,[5.7443225321e-06]
3,X21,[0.00871537074377]
4,X25,[0.00637521190534]


All the coefficients are small and close to 0. The coefficients on # of 311 complaints, # of ECB summons, Residential FAR Zoning, and # of DOB comlaints are positive, so an increase in these variables would result in an increase in the likelihood that illegal construction is occuring in that area. The coefficient on Residential Square Footage built is negative, and this makes sense since areas with greater residential square footage density should have more residents, who are vigilant to watch out for illegal construction or other activities, making the likelihood smaller. 

## Model Evaluation Using a Validation Set

We have trained and tested on the same set. 
Now we are going to instead split the data into a training set and a testing set.

In [150]:
#Split the data into training and test sets

X_train, X_test, y_train, y_test=train_test_split(model_train_X, model_train_Y, random_state=4)

In [151]:
#Fit logistic regression over training set

mireg = LogisticRegression()
mireg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [152]:
#Test accuracy over test set, print coefficients of logit

pred = mireg.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, pred)    
### Indicadores 
roc_auc = auc(fpr, tpr)
acc =  accuracy_score(model_train_Y, mireg.predict(model_train_X))

print ("Accuracy Score : %f" % acc)
print("Area under the ROC curve : %f" % roc_auc)

print "Coefficients"
print mireg.coef_

Accuracy Score : 0.551502
Area under the ROC curve : 0.904959
Coefficients
[[  7.10765348e-04   5.99787879e-03   3.30737427e-06   9.37129336e-03
    6.71519908e-03]]


The accuracy is 55%, which is the same as we experienced when training and predicting on the same data, but the area under the ROC curve is 0.90, which is not surprising because a model that correctly predicts half the data will perform well on a data where the mean likelihood is nearly 50%.

In [153]:
# The optimal cut off would be where tpr is high and fpr is low
# tpr - (1-fpr) is zero or near to zero is the optimal cut off point

i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(thresholds, index = i)})
roc.ix[(roc.tf-0).abs().argsort()[:1]]

# Plot tpr vs 1-fpr
fig, ax = pl.subplots()
pl.plot(roc['tpr'])
pl.plot(roc['1-fpr'], color = 'red')
pl.xlabel('1-False Positive Rate')
pl.ylabel('True Positive Rate')
pl.title('Receiver operating characteristic')
ax.set_xticklabels([])


[]

In [168]:
roc.head()

,1-fpr,fpr,tf,thresholds,tpr
0,1.000000,0.000000,-0.991071,0.994999,0.008929
1,1.000000,0.000000,-0.785714,0.882416,0.214286
2,0.991736,0.008264,-0.777450,0.881890,0.214286
3,0.991736,0.008264,-0.536378,0.758382,0.455357
4,0.983471,0.016529,-0.528114,0.757931,0.455357


The model is taking as a positive ECB hering predicted case all the probabilities above 61%.

In [170]:
predicted = mireg.predict(X_test)

In [172]:
pred = mireg.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, pred)    
### Indicadores 
roc_auc = auc(fpr, tpr)
acc =  accuracy_score(model_train_Y, mireg.predict(model_train_X))
print acc
print("Area under the ROC curve : %f" % roc_auc)


0.551502145923
Area under the ROC curve : 0.904959


In the next cells we can observe the confusion matrix and a classification report with other metrics.

In [176]:
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

[[ 10 111]
 [  0 112]]
             precision    recall  f1-score   support

        0.0       1.00      0.08      0.15       121
        1.0       0.50      1.00      0.67       112

avg / total       0.76      0.52      0.40       233



## Model Evaluation Using Cross-Validation

We are going to try 10-fold cross-validation.

In [181]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), model_train_X, model_train_Y, scoring='accuracy', cv=10)
print scores
print scores.mean()

[ 0.5106383   0.57446809  0.57446809  0.58510638  0.6344086   0.51612903
  0.74193548  0.49462366  0.5         0.5326087 ]
0.566438632091


## Using K Neighbors Clasifier Gives Higher Accuracy

In [186]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print metrics.accuracy_score(y_test, y_pred)

0.849785407725
